In [1]:
!pip install bitsandbytes --quiet
!pip install transformers --quiet
!pip install peft --quiet
!pip install accelerate --quiet
!pip install trl --quiet
!pip install datasets --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 21.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.4 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 30.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 2.0 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 85.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 348.0/348.0 kB 6.4 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 12.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are inst

In [2]:
import os
import pandas as pd
import torch
from datasets import load_dataset
from trl import SFTTrainer
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, TrainingArguments
from peft import LoraConfig, PeftModel, prepare_model_for_kbit_training

os.environ["WANDB_DISABLED"] = "true"

2025-05-17 21:21:01.450910: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747516861.668763      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747516861.730283      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
dataset = load_dataset("tatsu-lab/alpaca")["train"]

dataset[0]

README.md:   0%|          | 0.00/7.47k [00:00<?, ?B/s]

data/train-00000-of-00001-a09b74b3ef9c3b(…):   0%|          | 0.00/24.2M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/52002 [00:00<?, ? examples/s]

{'instruction': 'Give three tips for staying healthy.',
 'input': '',
 'output': '1.Eat a balanced diet and make sure to include plenty of fruits and vegetables. \n2. Exercise regularly to keep your body active and strong. \n3. Get enough sleep and maintain a consistent sleep schedule.',
 'text': 'Below is an instruction that describes a task. Write a response that appropriately completes the request.\n\n### Instruction:\nGive three tips for staying healthy.\n\n### Response:\n1.Eat a balanced diet and make sure to include plenty of fruits and vegetables. \n2. Exercise regularly to keep your body active and strong. \n3. Get enough sleep and maintain a consistent sleep schedule.'}

In [4]:
model_name_or_path = "/kaggle/input/llama-3.2/transformers/3b-instruct/1"

tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, trust_remote_code=True)
tokenizer.padding_side = "right"
tokenizer.pad_token = tokenizer.eos_token
tokenizer.add_eos_token = True

print(tokenizer.bos_token, tokenizer.eos_token)


<|begin_of_text|> <|eot_id|>


In [5]:
def format_chat_template(row):
    messages = [
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": row["instruction"]},
        {"role": "assistant", "content": row["output"]}
    ]
    
    chat = tokenizer.apply_chat_template(messages, tokenize=False)
    return {"text": chat}


format_chat_template(dataset[0])

{'text': '<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nYou are a helpful assistant.<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nGive three tips for staying healthy.<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\n1.Eat a balanced diet and make sure to include plenty of fruits and vegetables. \n2. Exercise regularly to keep your body active and strong. \n3. Get enough sleep and maintain a consistent sleep schedule.<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\n'}

In [6]:
dataset = dataset.shuffle(seed=42).select(range(11000))

processed_dataset = dataset.map(
    format_chat_template,
    num_proc= os.cpu_count(),
)

dataset = processed_dataset.train_test_split(test_size=0.1)

Map (num_proc=4):   0%|          | 0/11000 [00:00<?, ? examples/s]

In [7]:
compute_dtype = torch.bfloat16
bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=compute_dtype,
        bnb_4bit_use_double_quant=True
)

model = AutoModelForCausalLM.from_pretrained(
    model_name_or_path,
    trust_remote_code=True,
    quantization_config=bnb_config,
    device_map='auto',
)
model.config.use_cache = False
model.gradient_checkpointing_enable()

model = prepare_model_for_kbit_training(model)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

## Train

In [8]:
peft_config = LoraConfig(
    lora_alpha=64,
    lora_dropout=0.05,
    r=4,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules="all-linear"
)

In [9]:
training_arguments = TrainingArguments(
        output_dir="./llama3_sft/",
        do_eval=True,
        # eval_strategy="epoch",
        optim="paged_adamw_8bit",
        per_device_train_batch_size=8,
        gradient_accumulation_steps=2,
        per_device_eval_batch_size=8,
        logging_steps=10,
        learning_rate=8e-6,
        num_train_epochs=2,
        warmup_steps=3,
        lr_scheduler_type="linear",
        max_steps=200,  # demo temp
        # fp16=True, 
)

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [10]:
trainer = SFTTrainer(
        model=model,
        train_dataset=dataset['train'],
        eval_dataset=dataset['test'],
        peft_config=peft_config,
        args=training_arguments,
)

trainer.train()

Converting train dataset to ChatML:   0%|          | 0/9900 [00:00<?, ? examples/s]

Adding EOS to train dataset:   0%|          | 0/9900 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/9900 [00:00<?, ? examples/s]

Truncating train dataset:   0%|          | 0/9900 [00:00<?, ? examples/s]

Converting eval dataset to ChatML:   0%|          | 0/1100 [00:00<?, ? examples/s]

...

Tokenizing eval dataset:   0%|          | 0/1100 [00:00<?, ? examples/s]

Truncating eval dataset:   0%|          | 0/1100 [00:00<?, ? examples/s]

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Step,Training Loss
10,2.871600
20,2.576800
30,2.196500
40,2.024300
50,1.784300
60,1.618500
70,1.524300
80,1.448200
90,1.450400
100,1.406500


TrainOutput(global_step=200, training_loss=1.6418570566177368, metrics={'train_runtime': 3898.6987, 'train_samples_per_second': 0.821, 'train_steps_per_second': 0.051, 'total_flos': 9810597599281152.0, 'train_loss': 1.6418570566177368})

## Inference

In [11]:
sample = dataset["test"][0]
print(sample)


def format_chat_template(row):
    messages = [
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": row["instruction"]}
    ]    
    prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    return prompt

prompt = format_chat_template(sample)
prompt

{'instruction': 'Provide the necessary materials for the given project.', 'input': 'Build a birdhouse', 'output': 'Materials Needed for Building a Birdhouse:\n-Pieces of wood for the base, walls and roof of the birdhouse \n-Saw \n-Screws \n-Screwdriver \n-Nails \n-Hammer \n-Paint\n-Paintbrushes \n-Drill and bits \n-Gravel (optional)', 'text': '<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nYou are a helpful assistant.<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nProvide the necessary materials for the given project.<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\nMaterials Needed for Building a Birdhouse:\n-Pieces of wood for the base, walls and roof of the birdhouse \n-Saw \n-Screws \n-Screwdriver \n-Nails \n-Hammer \n-Paint\n-Paintbrushes \n-Drill and bits \n-Gravel (optional)<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\n'}


'<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nYou are a helpful assistant.<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nProvide the necessary materials for the given project.<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\n'

In [12]:
# sft_model = AutoModelForCausalLM.from_pretrained("./llama3_sft/")

sft_model = AutoModelForCausalLM.from_pretrained("/kaggle/input/llama-3.2/transformers/3b-instruct/1")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [13]:
inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

output = model.generate(
    **inputs,
    max_new_tokens=100,
    do_sample=True,
    temperature=0.7,
    pad_token_id=tokenizer.eos_token_id,
)

generated_text = tokenizer.decode(output[0], skip_special_tokens=False)
print(generated_text[len(prompt):].split("<|eot_id|>")[0]) 

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/torch/utils/checkpoint.py:87: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


The materials needed for this project are:
- A piece of wood
- A drill
- A hammer
- A saw
- A sandpaper
- Paint
- Paintbrushes
- A paint tray
